# Netcdf and Basemap

### Exercise:

Make a plot of the model temperature and wind over South America using Basemap. This plot must contains:
 * States boundarys (See the following link: http://basemaptutorial.readthedocs.io/en/latest/shapefile.html)
 * Meridians and parallels (Get http://matplotlib.org/basemap/users/examples.html)
 * Colorbar with a label
 * A title containing the variables and date

In [14]:
import numpy as np
from scipy.io import netcdf
from matplotlib import pyplot as plt
from mpl_toolkits.basemap import Basemap
from datetime import datetime, timedelta
%matplotlib inline

In [15]:
f = netcdf.netcdf_file('./data/r2_reanalysis.nc', 'r')

In [16]:
u = f.variables['u']
v = f.variables['v']
t = f.variables['t']

longitude = f.variables['lon'][:]
latitude = f.variables['lat'][:]
level = f.variables['level'][:]
time = f.variables['time']

In [17]:
def unpack(var):
    return var[:] * var.scale_factor + var.add_offset 

In [7]:
u = unpack(u)
v = unpack(v)
t = unpack(t)

In [8]:
def flip_grid(var, lons):
    fltr = lons >= 180
    # fltr =  [False False False ... True  True  True]
    newlons = np.concatenate(((lons - 360)[fltr], lons[~fltr]), axis=-1)
    # newlons = [-180 -177.5 -175 ... -5 -2.5 ] concatenated with [0 2.5 5 ... 175 177.5]
    # newlons = [-180 -177.5 -175 ... 175 177.5 180]
    if var.ndim == 2:
        newvar = np.concatenate((var[:, fltr], var[:, ~fltr]), axis=-1)
    elif var.ndim == 3:
        newvar = np.concatenate((var[:, :, fltr], var[:, :, ~fltr]), axis=-1)
    elif var.ndim == 4:
        newvar = np.concatenate((var[:, :, :, fltr], var[:, :, :, ~fltr]), axis=-1)        
        
    return newvar, newlons

In [9]:
u, newlon = flip_grid(u, longitude)
v, newlon = flip_grid(v, longitude)
t, newlon = flip_grid(t, longitude)

In [10]:
lons, lats = np.meshgrid(newlon, latitude)

In [11]:
def get_dates(time):
    splited_date = time.units.split()
    # splited_date = ['hours', 'since', '1800-1-1', '00:00:00']
    start_date_string = ' '.join(splited_date[-2:])
    # start_date_string = 1800-1-1 00:00:00
    # convert string into datetime object
    start_date = datetime.strptime(start_date_string, '%Y-%m-%d %H:%M:%S')
    
    dates = [start_date + timedelta(hours=i) for i in time[:]]
    return dates

In [12]:
dates = get_dates(time)

## JUST DO IT!